In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from tqdm import tqdm 
import warnings
warnings.filterwarnings('ignore')
import seaborn as sn
import time 
import matplotlib.animation as ani


dff = pd.read_excel('Datensätze.xlsx')
dff.replace(0, float('NaN'), inplace=True)
dff.dropna(inplace=True)


def Add_SBRatioRank(df, n):
# calculate and add Stock/Bond Ratio Rank
# first calculate all in a seperate 

    df['SB_Ratio'] = df['Stock_P']/df['Bond_P']
    df['SB_Ratio_Rank'] = (df['SB_Ratio']-df['SB_Ratio'].rolling(n).min())/(df['SB_Ratio'].rolling(n).max()-df['SB_Ratio'].rolling(n).min())

    return df


def Add_RiskPremiumRank(df, n):
# calculate risk premium Rank

    df['Risk_Premium'] = ((df['Earnings']/df['SPX_P']) - df['US10Y_Yield'])*-1
    df['Risk_Premium_Rank'] = (df['Risk_Premium']-df['Risk_Premium'].rolling(n).min())/(df['Risk_Premium'].rolling(n).max()-df['Risk_Premium'].rolling(n).min())
    return df


def Add_PutCallVolRank(df, n):
# calculate Put/Call Implied Volatility Rank



    df['SPX_Impl_Spread_P-C_Rank'] = -1*(df['SPX_Impl_Spread_P-C']-df['SPX_Impl_Spread_P-C'].rolling(n).min())/(df['SPX_Impl_Spread_P-C'].rolling(n).max()-df['SPX_Impl_Spread_P-C'].rolling(n).min())+1
    
    return df


def Add_HYCDSRank(df, n):
# calculate High Yield Credit Default Swap Rank
    
    df['HY_CDS_Rank'] = -1*(df['HY_CDS']-df['HY_CDS'].rolling(n).min())/(df['HY_CDS'].rolling(n).max()-df['HY_CDS'].rolling(n).min())+1
    
    return df


def Add_Vola_CurveRank(df,n):
# calculate Vola Curve Rank
    
    df['Vola_Slope_Rank'] = (df['Vola_Slope']-df['Vola_Slope'].rolling(n).min())/(df['Vola_Slope'].rolling(n).max()-df['Vola_Slope'].rolling(n).min())
    
    return df


def Add_DollarRank(df,n):
# calculate Dollar Rank

    df['Dollar_Rank'] = -1*(df['Dollar ']-df['Dollar '].rolling(n).min())/(df['Dollar '].rolling(n).max()-df['Dollar '].rolling(n).min())+1
    
    return df


def MA(serIn, n):
    
    if n == 0:
        serOut = serIn
    else:
        serOut = serIn.rolling(n).mean()
    return serOut


def do_Strat(dff, n1=250, n2=200, n3=350, n4=150, n5=200, n6=250, n7=120, a1=11, a2=11, a3=12, a4=0, a5=11, a6=14, a7=15, evaluate=True):

# do the strategy and chose 

# copy the dataframe to use 
    df = dff.copy()

# calculate future Returns
    df['1M nReturn'] = np.log(df['Stock_P'].shift(-21)/df['Stock_P'].shift(-1))
    df['3M nReturn'] = np.log(df['Stock_P'].shift(-61)/df['Stock_P'].shift(-1))
    df['6M nReturn'] = np.log(df['Stock_P'].shift(-121)/df['Stock_P'].shift(-1))
    df['Stock_MA'] = df.Stock_P.rolling(80).mean()

# calculate values 
    df = Add_SBRatioRank(df, n1)
    df = Add_RiskPremiumRank(df, n2)
    df = Add_PutCallVolRank(df, n3)
    df = Add_HYCDSRank(df, n4)
    df = Add_Vola_CurveRank(df, n5)
    df = Add_DollarRank(df,n6)
    
    df['SB_Ratio_Rank'] = MA(df['SB_Ratio_Rank'],a1)
    df['Risk_Premium_Rank'] = MA(df['Risk_Premium_Rank'], a2)
    df['Vola_Slope_Rank'] = MA(df['Vola_Slope_Rank'], a3)
    df['HY_CDS_Rank'] = MA(df['HY_CDS_Rank'], a4)
    df['SPX_Impl_Spread_P-C_Rank'] = MA(df['SPX_Impl_Spread_P-C_Rank'],a5)
    df['Dollar_Rank'] = MA(df['Dollar_Rank'], a6)

# create Score
    df['Score'] = df['SB_Ratio_Rank']+df['Risk_Premium_Rank']+df['Vola_Slope_Rank']*1.2+df['HY_CDS_Rank']+df['SPX_Impl_Spread_P-C_Rank']+df['Dollar_Rank']
    df['Score_MA'] = df['Score'].rolling(n7).mean()
    df['Score_Z'] = (df['Score']-df['Score_MA'])/df['Score'].rolling(n7).std()
    df['Score_Rank'] = (df['Score']-df['Score'].rolling(n7).min())/(df['Score'].rolling(n7).max()-df['Score'].rolling(n7).min())
    df['Score_Rank'] = MA(df['Score_Rank'],a7)

# drop NaN    
    if evaluate:
        df.dropna(inplace=True)

    return df

In [2]:
result = pd.DataFrame(columns=['n1', 
                               'n2', 
                               'n3', 
                               'n4', 
                               'n5',
                               'n6', 
                               'n7',
                               'a1', 
                               'a2',
                               'a3',
                               'a4',
                               'a5',
                               'a6',
                               'a7',
                               '6MS H',
                               '6MS',
                               '3MS H',
                               '3MS',
                               '6MB H',
                               '6MB', 
                               '3MB H',
                               '3MB',
                              ])

w = 0

for a in tqdm(np.linspace(100, 300, 3), leave=True):
    for b in tqdm(np.linspace(100, 300, 3), leave=True):
        for c in tqdm(np.linspace(100, 300, 3), leave=True):
            for d in np.linspace(100, 300, 3):
                for e in np.linspace(100, 300, 3):
                    for f in np.linspace(100, 300, 3):
                        for g in np.linspace(100,300,3):
                            for h in np.linspace(8, 16, 2):
                                for i in np.linspace(8, 16, 2):
                                    for j in np.linspace(8, 16, 2):
                                        for k in np.linspace(8, 16, 2):
                                            for l in np.linspace(8, 16, 2):
                                                for m in np.linspace(8, 16, 2):
                                                    for n in np.linspace(8, 16, 2):

                                                        df = do_Strat(dff, int(a), int(b), int(c), int(d), int(e), int(f), int(g), int(h), int(i),int(j) , int(k), int(l), int(m), int(n))

                                                        df['SELL'] = (df['Score_Rank'] > 0.85) & (df['Score_Rank'].shift(1) < 0.85)
                                                        df['BUY'] = (df['Score_Rank'] < (0.15)) & (df['Score_Rank'].shift(1) > (0.15))

                                                        o = len([num for num in (df['6M nReturn']*df['SELL']) if num < 0])/len([num for num in (df['6M nReturn']*df['SELL']) if num != 0])
                                                        p = ((df['6M nReturn']*df['SELL']*-1).sum())/(df['SELL'].sum())

                                                        q = len([num for num in (df['3M nReturn']*df['SELL']) if num < 0])/len([num for num in (df['3M nReturn']*df['SELL']) if num != 0])
                                                        r = ((df['3M nReturn']*df['SELL']*-1).sum())/(df['SELL'].sum())

                                                        s = len([num for num in (df['6M nReturn']*df['BUY']) if num < 0.05])/len([num for num in (df['6M nReturn']*df['BUY']) if num != 0])
                                                        t = ((df['6M nReturn']*df['BUY']*1).sum())/(df['BUY'].sum())

                                                        u = len([num for num in (df['3M nReturn']*df['BUY']) if num < 0.025])/len([num for num in (df['3M nReturn']*df['BUY']) if num != 0])
                                                        v = ((df['3M nReturn']*df['BUY']*1).sum())/(df['BUY'].sum())


                                                        result = result.append(pd.Series([0]), ignore_index=True)

                                                        result.iloc[w, 0] = a
                                                        result.iloc[w, 1] = b
                                                        result.iloc[w, 2] = c
                                                        result.iloc[w, 3] = d
                                                        result.iloc[w, 4] = e
                                                        result.iloc[w, 5] = f
                                                        result.iloc[w, 6] = g
                                                        result.iloc[w, 7] = h
                                                        result.iloc[w, 8] = i
                                                        result.iloc[w, 9] = j
                                                        result.iloc[w, 10] = k
                                                        result.iloc[w, 11] = l
                                                        result.iloc[w, 12] = m
                                                        result.iloc[w, 13] = n
                                                        result.iloc[w, 14] = o
                                                        result.iloc[w, 15] = p
                                                        result.iloc[w, 16] = q
                                                        result.iloc[w, 17] = r
                                                        result.iloc[w, 18] = s
                                                        result.iloc[w, 19] = t
                                                        result.iloc[w, 20] = u
                                                        result.iloc[w, 21] = v

                                                        w+=1

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [04:44<09:28, 284.36s/it]

 67%|██████▋   | 2/3 [09:45<04:49, 289.29s/it]

100%|██████████| 3/3 [16:16<00:00, 325.40s/it]

 33%|███▎      | 1/3 [16:16<32:32, 976.20s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [08:32<17:04, 512.35s/it]

 67%|██████▋   | 2/3 [19:07<09:09, 549.33s/it]

100%|██████████| 3/3 [32:35<00:00, 651.88s/it]

 67%|██████▋   | 2/3 [48:51<21:10, 1270.03s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [15:54<31:48, 954.04s/it]

 67%|██████▋   | 2/3 [34:35<16:44, 1004.41s/it]

100%|██████████| 3/3 [56:29<00:00, 1129.99s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [25:32<51:04, 1532.14s/it]

 67%|██████▋   | 2/3 [54:11<26:28, 1588.45s/it]

100%|██████████| 3/3 [1:25:43<00:00, 1714.59s/it]

 33%|███▎      | 1/3 [1:25:43<2:51:27, 5143.78s/it]

  0%|          | 0/3 [00:00<?, ?i

In [3]:
result.to_excel('3.xlsx')